In [928]:
import pandas as pd
import numpy as np
from geopy.geocoders import GoogleV3, Nominatim
from geopy.distance import vincenty, great_circle
import math

In [929]:
US_trails = pd.read_csv('../US_trails_engineered.csv')

In [930]:
US_trails.shape

(20691, 18)

In [931]:
US_trails.columns

Index(['name', 'location', 'difficulty', 'length', 'ascent', 'descent',
       'stars', 'latitude', 'longitude', 'summary', 'url',
       'difficulty_encoded', 'category', 'type_Featured Ride', 'type_Trail',
       'city/town', 'state', 'length_range'],
      dtype='object')

In [932]:
US_trails.index

RangeIndex(start=0, stop=20691, step=1)

In [933]:
geolocator = GoogleV3()

In [934]:
#location = geolocator.geocode("1182 Shepherds Lane NE")
location = geolocator.geocode("2044 South Huron Street Denver")
#location = geolocator.geocode('University of Colorado')
#location = geolocator.geocode('Iowa City, Iowa')
location.address

'2044 S Huron St, Denver, CO 80223, USA'

In [935]:
lat_lon = (location.latitude, location.longitude)
lat_lon

(39.6795193, -104.9969322)

In [936]:
# Distances are measured in miles.
# Longitudes and latitudes are measured in degrees.
# Earth is assumed to be perfectly spherical.

earth_radius = 3960.0
degrees_to_radians = math.pi/180.0
radians_to_degrees = 180.0/math.pi

def change_in_latitude(miles):
    "Given a distance north, return the change in latitude."
    return (miles/earth_radius)*radians_to_degrees

def change_in_longitude(latitude, miles):
    "Given a latitude and a distance west, return the change in longitude."
    # Find the radius of a circle around the earth at given latitude.
    r = earth_radius*math.cos(latitude*degrees_to_radians)
    return (miles/r)*radians_to_degrees

In [937]:
#I want to drive 15 miles north, or 15 miles west
m = 15
lat_diff = change_in_latitude(m)
lat_diff

0.21702946785258456

In [938]:
lon_diff = change_in_longitude(lat_lon[0], m)
lon_diff

0.2819927468892867

In [939]:
#make a function that takes in a location, a desired crow fly distance radius, and turns it into a lat/lon difference
#add/subtract that distance in lat/long from where they are to get intervals of lat/lon within to return all the trails
#that are in that interval range
#maybe return a column with the number of miles in a crow's distance(have a disclaimer about the range of miles
#this could be driving)

In [940]:
lat_min = lat_lon[0] - lat_diff
lat_min

39.46248983214742

In [941]:
lat_max = lat_lon[0] + lat_diff
lat_max

39.896548767852586

In [942]:
lon_min = lat_lon[1] - lon_diff
lon_min

-105.27892494688929

In [943]:
lon_max = lat_lon[1] + lon_diff
lon_max

-104.71493945311072

In [944]:
lat_range = (lat_min, lat_max)
lat_range

(39.46248983214742, 39.896548767852586)

In [945]:
lon_range = (lon_min, lon_max)
lon_range

(-105.27892494688929, -104.71493945311072)

In [946]:
df = US_trails[(US_trails['latitude'] >= lat_range[0]) & (US_trails['latitude'] <= lat_range[1])] #and 
df.shape

(866, 18)

In [947]:
df.index

Int64Index([ 9933, 10081, 10082, 10083, 10086, 10087, 10089, 10094, 10098,
            10105,
            ...
            16596, 16597, 16610, 16638, 16750, 16751, 16753, 16754, 16755,
            16757],
           dtype='int64', length=866)

In [948]:
df = df[(df['longitude'] >= lon_range[0]) & (df['longitude'] <= lon_range[1])]
df.shape

(112, 18)

In [949]:
df.index

Int64Index([10472, 10479, 10482, 10497, 10500, 10502, 10504, 10505, 10507,
            10511,
            ...
            11510, 11511, 11513, 11514, 15684, 16547, 16580, 16594, 16596,
            16610],
           dtype='int64', length=112)

In [950]:
df.columns

Index(['name', 'location', 'difficulty', 'length', 'ascent', 'descent',
       'stars', 'latitude', 'longitude', 'summary', 'url',
       'difficulty_encoded', 'category', 'type_Featured Ride', 'type_Trail',
       'city/town', 'state', 'length_range'],
      dtype='object')

In [951]:
df['latitude'].min()

39.489800000000002

In [952]:
df['latitude'].max()

39.887900000000002

In [953]:
df['longitude'].min()

-105.2764

In [954]:
df['longitude'].max()

-104.82170000000001

In [955]:
df['name']

10472               Lair O' the Bear - Bear Creek Trail
10479            Mount Falcon and Lair O' the Bear Loop
10482                             Meyer Ranch Full Loop
10497    Dakota Ridge / Red Rocks loop from Zorro trail
10500                                 Deer Creek Canyon
10502                        Green Mountain Novice Loop
10504                                  Hildebrand Ranch
10505                  Green Mountain Intermediate Loop
10507        Mount Falcon - The best of the singletrack
10511                    Bear Creek - Dakota Ridge Loop
10512                                    Front Range 30
10513                                     XC Loop Trail
10514                                    Mt Carbon Loop
10515                                Dakota Ridge Trail
10516                                   Red Rocks Trail
10517                                      Castle Trail
10518                                    Parmalee Trail
10519                              Box o' Rox Ex

In [956]:
#dfd.loc[dfd.index[[0, 2]], 'A']
new_lat_lon = (df.loc[11427]['latitude'], df.loc[11427]['longitude'])
new_lat_lon

(39.717799999999997, -105.2302)

In [957]:
lat_lon

(39.6795193, -104.9969322)

In [958]:
new_lat_lon

(39.717799999999997, -105.2302)

In [959]:
vincenty(lat_lon, new_lat_lon).miles

12.709150685596637

In [960]:
df.index

Int64Index([10472, 10479, 10482, 10497, 10500, 10502, 10504, 10505, 10507,
            10511,
            ...
            11510, 11511, 11513, 11514, 15684, 16547, 16580, 16594, 16596,
            16610],
           dtype='int64', length=112)

In [961]:
df.loc[10472]

name                                Lair O' the Bear - Bear Creek Trail
location                                              Genesee, Colorado
difficulty                                                         Blue
length                                                             12.2
ascent                                                             1673
descent                                                           -1673
stars                                                                 4
latitude                                                        39.6676
longitude                                                      -105.257
summary               Mostly flowy singletrack, with a few small cli...
url                   https://www.mtbproject.com/trail/703097/lair-o...
difficulty_encoded                                                    3
category                                                  Featured Ride
type_Featured Ride                                              

In [962]:
'''
def get_vincenty(num):
    new_lat_lon = (df.loc[num]['latitude'], df.loc[num]['longitude'])
    return round(vincenty(lat_lon, new_lat_lon).miles,1)

df['miles_away'] = df.index.map(get_vincenty)
#this was a good idea but I kept having errors with the index
'''

"\ndef get_vincenty(num):\n    new_lat_lon = (df.loc[num]['latitude'], df.loc[num]['longitude'])\n    return round(vincenty(lat_lon, new_lat_lon).miles,1)\n\ndf['miles_away'] = df.index.map(get_vincenty)\n#this was a good idea but I kept having errors with the index\n"

In [963]:
US_trails['difficulty'].value_counts()

Blue            6782
Green/Blue      4560
Blue/Black      3491
Green           3100
Black           2320
Double Black     438
Name: difficulty, dtype: int64

In [964]:
US_trails['length_range'].value_counts()

0-5       16279
5-10       2381
10-15       995
15-20       452
20-25       251
30-50       149
25-30       112
50-100       49
100+         23
Name: length_range, dtype: int64

In [965]:
US_trails.dtypes

name                   object
location               object
difficulty             object
length                float64
ascent                  int64
descent                 int64
stars                 float64
latitude              float64
longitude             float64
summary                object
url                    object
difficulty_encoded      int64
category               object
type_Featured Ride      int64
type_Trail              int64
city/town              object
state                  object
length_range           object
dtype: object

In [966]:
def get_vincenty(row, *args):
    new_lat_lon = (row['latitude'], row['longitude'])    
    #return round(vincenty(args[0], new_lat_lon).miles,1)
    return round(vincenty(args[0], new_lat_lon).miles,1)

#df['miles_away'] = df.apply(get_vincenty, axis = 1)

In [967]:
def cold_start(start, miles, length_range = None, difficulty = None):
    #these are units of measurement that we will need in determining our lat/lon change
    earth_radius = 3960.0
    degrees_to_radians = math.pi/180.0
    radians_to_degrees = 180.0/math.pi
    
    #the geolocator starts an instance of Google's API so we can get the lat/lon of the starting location
    geolocator = GoogleV3()
    location = geolocator.geocode(start)
    location_lat_lon = (location.latitude, location.longitude)
    print (location.address)
    
    '''
    Convert the miles into a latitude difference: since an arc length(in miles) is represented by miles = radius * angle(in radians),
    then the latitude angle can be solved by angle = miles/earth_radius and then converted into degrees with radians_to_degrees.
    Since the longitude conversion of miles into degrees depends on the latitude, for longitude miles = earth_radius * theta * cos(lat_degree)
    where cos(lat_degree) is the proportion of the circle of latitude to the circle around the equator with the earth's radius.
    This makes sense because if we are at the equator, the radius of the latitude circle is equal to the radius of the earth;
    therefore cos(latitude angle) = 1, making the longitude miles = earth_radius * longitude_angle.  To solve for the 
    longitude angle, we get longitude_angle = miles/(earth_radius*cos(longitude_angle)).
    '''
    lat_diff = (miles/earth_radius)*radians_to_degrees
    r = earth_radius*math.cos(location_lat_lon[0]*degrees_to_radians)
    lon_diff = (miles/r)*radians_to_degrees
    
    #we now add/subtract the lat_diff and lon_diff in degrees from the lat/lon of our current location 
    #to find the range of lat and lon degrees from which we'll take our trails
    lat_range = (location_lat_lon[0] - lat_diff, location_lat_lon[0] + lat_diff)
    lon_range = (location_lat_lon[1] - lon_diff, location_lat_lon[1] + lon_diff)
    
    #we know create a datraframe that only includes trails within our lat and lon ranges
    df = US_trails[(US_trails['latitude'] >= lat_range[0]) & (US_trails['latitude'] <= lat_range[1])]
    df = df[(df['longitude'] >= lon_range[0]) & (df['longitude'] <= lon_range[1])]
    #could possibly put this in one line but that would be long
  
    #if a length_range for desired trails is given, we limit our dataframe to only include trails within that range
    if length_range == '0-5':
        df = df[df['length_range'] == '0-5']
    if length_range == '5-10':
        df = df[df['length_range'] == '5-10']
    if length_range == '10-15':
        df = df[df['length_range'] == '10-15']
    if length_range == '15-20':
        df = df[df['length_range'] == '15-20']
    if length_range == '20-25':
        df = df[df['length_range'] == '20-25']
    if length_range == '25-30':
        df = df[df['length_range'] == '25-30']
    if length_range == '30-50':
        df = df[df['length_range'] == '30-50']
    if length_range == '50-100':
        df = df[df['length_range'] == '50-100']
    if length_range == '100+':
        df = df[df['length_range'] == '100+']
        
    #if a difficulty for desired trails is given, we limit our dataframe to only include trails that have that difficulty
    if difficulty == 'Green':
        difficulties = ['Green', 'Green/Blue']
        df = df[df['difficulty'].isin(difficulties)]
    if difficulty == 'Blue':
        difficulties = ['Green/Blue', 'Blue', 'Blue/Black']
        df = df[df['difficulty'].isin(difficulties)]
    if difficulty == 'Black':
        difficulties = ['Blue/Black', 'Black', 'Double Black']
        df = df[df['difficulty'].isin(difficulties)]
    
    #if there's no trails that fit all these requirements, we must give a warning so they don't wonder why there's an empty df
    if df.shape[0] == 0:
        return "There are no trails that meet your requirements. Try expanding your search."
    
    #add a column to our df that includes the miles away that each trail is from our location using the vincenty distance measurement
    else:
        #df['miles_away'] = df.index.map(get_vincenty)
        df['miles_away'] = df.apply(get_vincenty, axis = 1, args = (location_lat_lon,))
        columns_to_output = ['name', 'location', 'stars', 'difficulty', 'length', 'ascent', 'descent',
            'category', 'miles_away', 'summary', 'url']
        return df[columns_to_output].sort_values('miles_away')
    
'''
make it optional to put in a distance_length_range max(might need to encode these so I can return the trails that have 
less than that range, or more), and might need to make a column for location, state
also do this for difficulty
#if you want to ride 15 - 20, just give back 15 - 20
#if you want mostly green, do green and green/blue
#if you want mostly blue, do green/blue, blue, blue/black
#if you want mostly black, do blue/black, black, double black
'''

'\nmake it optional to put in a distance_length_range max(might need to encode these so I can return the trails that have \nless than that range, or more), and might need to make a column for location, state\nalso do this for difficulty\n#if you want to ride 15 - 20, just give back 15 - 20\n#if you want mostly green, do green and green/blue\n#if you want mostly blue, do green/blue, blue, blue/black\n#if you want mostly black, do blue/black, black, double black\n'

In [968]:
#cold_start('2044 South Huron Street Denver', 15, difficulty = 'Black')
cold_start('2044 South Huron Street Denver', 15, length_range = '0-5', difficulty = 'Black')

2044 S Huron St, Denver, CO 80223, USA


,name,location,stars,difficulty,length,ascent,descent,category,miles_away,summary,url
10519,Box o' Rox Extension,"West Pleasant View, Colorado",4.2,Blue/Black,0.6,0,-150,Trail,10.4,"A COMBA-built trail offering flow, berms, jump...",https://www.mtbproject.com/trail/7027217/box-o...
10520,South Dakota Ridge Trail,"Morrison, Colorado",4.8,Black,0.6,46,-95,Trail,10.4,A lollipop extension of the Dakota Ridge Trail.,https://www.mtbproject.com/trail/7030099/south...
10670,Zorro,"Morrison, Colorado",3.0,Blue/Black,1.0,0,-383,Trail,10.8,A singletrack connector between Dakota Ridge a...,https://www.mtbproject.com/trail/163982/zorro
10517,Castle Trail,"Morrison, Colorado",3.6,Blue/Black,3.9,1740,0,Trail,10.9,Not a super technical climb but nonetheless de...,https://www.mtbproject.com/trail/702276/castle...
10515,Dakota Ridge Trail,"Morrison, Colorado",4.3,Black,2.7,349,-706,Trail,11.1,"A technical, rocky, fast trail.",https://www.mtbproject.com/trail/605246/dakota...
10584,Morrison Slide Trail,"Morrison, Colorado",3.3,Black,1.4,285,-360,Trail,11.2,An alternate route off of the Red Rocks Trail.,https://www.mtbproject.com/trail/656058/morris...
11434,Pick N' Sledge,"Golden, Colorado",3.6,Blue/Black,1.5,713,-18,Trail,12.1,An alternate climb up Apex trail.,https://www.mtbproject.com/trail/603826/pick-n...
10598,Plymouth Creek Trail,"Ken Caryl, Colorado",3.5,Black,2.3,1092,-56,Trail,12.5,The main trail connecting you to the Red Mesa ...,https://www.mtbproject.com/trail/1366318/plymo...
10539,Devil's Elbow,"Indian Hills, Colorado",4.1,Blue/Black,0.9,91,-156,Trail,12.5,A nice little loop with some great views and i...,https://www.mtbproject.com/trail/4442994/devil...
10561,Old Ute,"Indian Hills, Colorado",4.0,Blue/Black,0.5,95,-80,Trail,12.6,A short loop with some interesting rock features.,https://www.mtbproject.com/trail/4443458/old-ute


In [969]:
cold_start('1182 Shepherds Lane Atlanta', 10)

1182 Shepherds Ln NE, Atlanta, GA 30324, USA


,name,location,stars,difficulty,length,ascent,descent,category,miles_away,summary,url
3170,Sope Creek,"Vinings, Georgia",4.1,Green/Blue,8.8,537,-525,Trail,8.7,Sope Creek has a few miles of gravel and some ...,https://www.mtbproject.com/trail/5545346/sope-...
2473,Silver Comet Trail,"Smyrna, Georgia",3.0,Green,63.5,1797,-1837,Trail,9.8,Needs Adoption!,https://www.mtbproject.com/trail/7004601/silve...


In [970]:
cold_start('690 North Pearl Street Denver', 15)

690 Pearl St, Denver, CO 80203, USA


,name,location,stars,difficulty,length,ascent,descent,category,miles_away,summary,url
10513,XC Loop Trail,"Englewood, Colorado",3.0,Green/Blue,1.7,130,-130,Featured Ride,3.3,"A fast, flowy XC loop (clockwise) with a littl...",https://www.mtbproject.com/trail/7032397/xc-lo...
16610,Outer Loop,"Wheat Ridge, Colorado",2.0,Blue,2.6,53,-55,Trail,6.8,"An easy mix of paved path, singletrack, and do...",https://www.mtbproject.com/trail/7030025/outer...
10701,Blackmer Lake Loop,"Cherry Hills Village, Colorado",2.5,Green,1.0,34,-41,Trail,6.9,A short loop around Blackmer Lake.,https://www.mtbproject.com/trail/7002900/black...
11514,Perimeter Trail,"Commerce City, Colorado",2.0,Green,22.2,333,-332,Trail,7.6,"A flat, mostly-gravel loop with cool wildlife.",https://www.mtbproject.com/trail/7022012/perim...
10557,Prairie Loop,"Centennial, Colorado",3.7,Green/Blue,1.0,50,-52,Trail,8.0,The second loop of Village Greens North Mounta...,https://www.mtbproject.com/trail/7003521/prair...
10564,Hill Loop,"Centennial, Colorado",3.5,Green/Blue,0.5,50,-49,Trail,8.1,A fun loop with easy and technical options thr...,https://www.mtbproject.com/trail/7003520/hill-...
10639,Prairie Loop to Trailhead,"Centennial, Colorado",3.7,Green,0.6,48,-2,Trail,8.1,Get back to the car after lapping Prairie Loop,https://www.mtbproject.com/trail/7003522/prair...
10699,Beginner Loop to Hill Loop,"Centennial, Colorado",2.5,Green,0.2,0,-30,Trail,8.2,A quick gravel trail with a few obstacles to g...,https://www.mtbproject.com/trail/7003517/begin...
10502,Green Mountain Novice Loop,"West Pleasant View, Colorado",3.1,Green/Blue,5.9,983,-995,Featured Ride,9.5,A relatively quick loop of great singletrack w...,https://www.mtbproject.com/trail/3638243/green...
10652,Hayden Trail,"Morrison, Colorado",3.3,Blue,1.7,629,-13,Trail,9.7,A good connector from the Green Mountain Loop ...,https://www.mtbproject.com/trail/7003415/hayde...


In [971]:
cold_start('Iowa City, Iowa', 150)

Iowa City, IA, USA


,name,location,stars,difficulty,length,ascent,descent,category,miles_away,summary,url
13820,Clear Creek Greenway,"Coralville, Iowa",3.0,Green/Blue,2.4,223,-226,Trail,3.6,Winding singletrack between the Clear Creek pa...,https://www.mtbproject.com/trail/7015001/clear...
13815,205,"North Liberty, Iowa",4.0,Blue,0.4,69,-78,Trail,6.4,Fully in the pines with plenty of exposed root...,https://www.mtbproject.com/trail/4606280/205
13814,301,"North Liberty, Iowa",4.0,Green,1.1,167,-207,Trail,6.4,Great view of the lake!,https://www.mtbproject.com/trail/4612447/301
13817,203,"North Liberty, Iowa",4.0,Blue,0.5,54,-54,Trail,6.5,Amp up the roots and amp up the cardio with 20...,https://www.mtbproject.com/trail/4607377/203
13818,302,"North Liberty, Iowa",4.0,Blue,0.3,35,-33,Trail,6.5,Winding climbing,https://www.mtbproject.com/trail/4610767/302
13802,204 - Hell Trail,"North Liberty, Iowa",5.0,Black,0.8,129,-99,Trail,6.5,"Makes you say, ""Aw, hell!""",https://www.mtbproject.com/trail/4792388/204-h...
13813,202,"North Liberty, Iowa",4.0,Blue,0.2,41,-13,Trail,6.6,Amp up the roots and amp up the cardio with 20...,https://www.mtbproject.com/trail/4606834/202
13816,303,"North Liberty, Iowa",4.0,Blue,0.5,109,-66,Trail,6.6,A technical section sneaks up near the end.,https://www.mtbproject.com/trail/4611318/303
13812,201,"North Liberty, Iowa",4.0,Green,1.3,189,-187,Trail,6.7,"An entertaining mix of twists and turns, ups a...",https://www.mtbproject.com/trail/4605688/201
13811,304,"North Liberty, Iowa",4.0,Black,0.8,56,-82,Trail,6.8,Gets its black rating for some steep and rooty...,https://www.mtbproject.com/trail/4611876/304


In [972]:
US_trails.loc[10737]
(39.49083302147416, 43.83142237852584)
(-94.43516851192199, -88.62516808807801)

(-94.43516851192199, -88.62516808807801)